In [25]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
flights = spark.read.csv('flights_small.csv', header=True)
planes = spark.read.csv('planes.csv', header=True)

In [47]:
flights.createOrReplaceTempView("flights")
planes.createOrReplaceTempView("planes")
#flights = flights.dropna(how="any")
flights.show()

+-------+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|yearOrg|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+-------+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|   2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|   2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|   2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|   2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|   2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|   2014|    1| 

In [16]:
#Quin és l’avió («tailnum») amb major petjada de carboni.
#Quin és l’avió («tailnum») amb menor petjada de carboni. ¿ha estat temps sense volar?
#Quina és la petjada promig i la petjada mitjana segons «tailnum».
#Si hi aprecieu diferència notable a què ho atribuiu?
from pyspark.sql.functions import min, max, avg
flights.filter('air_time != "NA"')
flights= flights.withColumn('air_timeInt', flights['air_time'].cast('int'))
flights.select(max("air_timeInt"),min("air_timeInt"),avg("air_timeInt")).show()

+----------------+----------------+------------------+
|max(air_timeInt)|min(air_timeInt)|  avg(air_timeInt)|
+----------------+----------------+------------------+
|             409|              20|152.88423173803525|
+----------------+----------------+------------------+



In [24]:
#Quina és l’aerolinia amb major petjada de carboni.
flights.groupBy("carrier").sum("air_timeInt").orderBy("sum(air_timeInt)",ascending = False).show(1)
#5. Quina és l’aerolinia amb menor petjada de carboni.
flights.groupBy("carrier").sum("air_timeInt").orderBy("sum(air_timeInt)",ascending = True).show(1)

+-------+----------------+
|carrier|sum(air_timeInt)|
+-------+----------------+
|     AS|          594898|
+-------+----------------+
only showing top 1 row

+-------+----------------+
|carrier|sum(air_timeInt)|
+-------+----------------+
|     VX|           19990|
+-------+----------------+
only showing top 1 row



In [33]:
flights=flights.withColumnRenamed("year", "yearOrg")
flightsJoin = flights.join(planes, on='tailnum',how='left')

flightsJoin.filter('air_time != "NA"')
flightsJoin= flightsJoin.withColumn('air_timeInt', flights['air_time'].cast('int'))

#6. Quin és el fabricant d’avions amb major petjada de carboni.ç
flightsJoin.groupBy("manufacturer").sum("air_timeInt").orderBy("sum(air_timeInt)",ascending = False).show(1)
#7. Quin és el fabricants d’avions amb menor petjada de carboni.
flightsJoin.groupBy("manufacturer").sum("air_timeInt").orderBy("sum(air_timeInt)",ascending = True).show(1)

+------------+----------------+
|manufacturer|sum(air_timeInt)|
+------------+----------------+
|      BOEING|         1044572|
+------------+----------------+
only showing top 1 row

+-------------+----------------+
| manufacturer|sum(air_timeInt)|
+-------------+----------------+
|BARKER JACK L|             131|
+-------------+----------------+
only showing top 1 row



In [34]:
#Quin és el model d’avions amb major petjada de carboni.
flightsJoin.groupBy("model").sum("air_timeInt").orderBy("sum(air_timeInt)",ascending = False).show(1)
#9. Quin és el model d’avions amb menor petjada de carboni.
flightsJoin.groupBy("model").sum("air_timeInt").orderBy("sum(air_timeInt)",ascending = True).show(1)

+-------+----------------+
|  model|sum(air_timeInt)|
+-------+----------------+
|737-890|          260964|
+-------+----------------+
only showing top 1 row

+-------+----------------+
|  model|sum(air_timeInt)|
+-------+----------------+
|737-3T5|             104|
+-------+----------------+
only showing top 1 row



In [37]:
#10. Fes una taula de petjada de carboni en funció dels anys de fabricació dels avions
flightsJoin.groupBy("year").sum("air_timeInt").orderBy("sum(air_timeInt)",ascending = False).show()

+----+----------------+
|year|sum(air_timeInt)|
+----+----------------+
|2001|          134090|
|2013|          106472|
|null|          102798|
|2006|           89694|
|2008|           88640|
|2007|           86854|
|1999|           86733|
|2000|           79398|
|2014|           72769|
|2012|           71847|
|1998|           71523|
|2009|           59339|
|2003|           55231|
|2002|           47505|
|2005|           42462|
|1992|           39326|
|2010|           39225|
|2011|           34904|
|1997|           33159|
|2004|           32899|
+----+----------------+
only showing top 20 rows



In [54]:
#11. Quina és l’aerolinia amb major petjada de carboni normalitzat al número de seients disponible a cada vol.

#flightsJoin.groupBy("seats").sum("air_timeInt")/().show()
query2 = 'SELECT f.carrier, sum(f.air_time)/sum(p.seats)  as calc FROM flights f , planes p GROUP BY f.carrier ORDER BY calc DESC'
flights_counts=spark.sql(query2).show(1)

+-------+------------------+
|carrier|              calc|
+-------+------------------+
|     HA|1.9008680219640914|
+-------+------------------+
only showing top 1 row

